In [2]:
from pixelmed_calc.medical_imaging.RadiologyComponents.tools.ith_score import batch_compute_ith_scores

directory_path = r"D:\pixelmedAI\function_pm\note1—影像组学\生境影像\molan_submask"
csv_filename = "ith_scores.csv"

    # Use the "cluster" method to replicate calITHscore behavior
res = batch_compute_ith_scores(
        directory_path=directory_path,
        csv_filename=csv_filename,
        method="cluster",#regionprops,cluster
        min_area=100,
        thresh=1
    )
print(f"Results saved to {csv_filename}. Count: {len(res)}")

Results saved to ith_scores.csv. Count: 10


# 两个 ITH 计算方法的公式

下面把你整理后的两个函数各自对应的数学公式写出来，便于文档或论文使用。记号尽量与代码语义一致。

## 方法一：regionprops 版本（基于连通域的填充面积比）

设标签图经过连通域分割得到区域集合 $\{R_i\}_{i=1}^{M}$。对每个区域：
- $\ n_i = \text{area}(R_i)$：区域体素数（面积/体积）
- $\ s_i = \text{filled\_area}(R_i)$：区域“孔洞填充后”的体素数

定义：
$\
S \;=\; \sum_{i=1}^{M} \frac{s_i}{n_i}
$

ITH 分数：
$\
\text{ITH}_{\text{regionprops}} \;=\; \frac{10}{\ln\!\big(1 + S\big)}
\;=\;
\frac{10}{\ln\!\left(1 + \sum_{i=1}^{M} \frac{s_i}{n_i}\right)}$


- **说明**：当区域存在孔洞或不规则时，\(s_i > n_i\) 的程度越大，\(S\) 越大，\(\ln(1+S)\) 越大，从而 $\text{ITH}$ 越小。该分数不在 \([0,1]\) 内。

## 方法二：cluster-map 版本（按标签逐类的连通性碎片化度量）

设标签图中非背景标签集合为 $\mathcal{L} = \{ \ell \in \mathbb{N} \mid \ell > 0, \ell \text{ 出现在图中} \}$。

- 记总体阳性体素数（所有正标签的体素总数）为：
$\
\text{size} \;=\; \sum_{x} \mathbf{1}\big(\text{label}(x) > 0\big)
$

- 对每个标签 $\ell \in \mathcal{L}$，在其二值掩模上（给定连接性，例如 8-邻域/26-邻域）求连通域个数和各连通域面积。根据总规模与阈值规则，过滤过小连通域；设过滤后：
  - $\text{num\_regions}_\ell$：该标签剩余连通域数
  - $\text{max\_area}_\ell$：该标签中最大连通域的体素数

定义中间量：
$\
A \;=\; \sum_{\ell \in \mathcal{L}^\star} \frac{\text{max\_area}_\ell}{\text{num\_regions}_\ell}
$
其中 $\mathcal{L}^\star \subseteq \mathcal{L}$为过滤后仍然有效的标签集合

ITH 分数：
$\
\text{ITH}_{\text{cluster}} \;=\; 1 \;-\; \frac{A}{\text{size}}
\;=\;
1 \;-\; \frac{1}{\text{size}} \sum_{\ell \in \mathcal{L}^\star} \frac{\text{max\_area}_\ell}{\text{num\_regions}_\ell}
$


# 参数含义：`min_area=100`, `thresh=1`

这两个参数出现在 cluster-map 版本的 ITH 计算中，用来在“总体肿瘤体素数较小”时，过滤掉极小的连通域（噪声），以避免它们放大碎片化程度。

## 含义与作用

- **`min_area=100`**
  - 含义：总体阳性体素数的阈值，即 `size = sum(label_map > 0)` 的门槛。
  - 作用：当总体肿瘤（所有正标签体素）数量 `size` 小于或等于 `min_area` 时，启用更严格的小连通域过滤规则（见下）。

- **`thresh=1`**
  - 含义：在“小样本模式”下判定小连通域的面积阈值。
  - 作用：
    - 若 `size <= min_area`（小样本模式），则会剔除面积 `<= thresh` 的连通域（默认就是剔除面积为 1 的小点）。
    - 若 `size > min_area`（大样本模式），代码固定剔除面积 `<= 1` 的连通域（点噪声），与 `thresh` 无关。

## 代码中的具体行为

- 计算总阳性体素数：`size = np.sum(label_map > 0)`
- 对每个标签的每个连通域，按两种模式过滤：
  - **小样本模式**（`size <= min_area`）：
    - 若组件面积 `<= thresh`：从计数中移除（`num_regions -= 1`），并在极端情况下设置跳过该标签（`flag = 0`）
    - 其余组件参与最大面积 `max_area` 的统计
  - **大样本模式**（`size > min_area`）：
    - 若组件面积 `<= 1`：从计数中移除（固定阈值 1）
    - 其余组件参与 `max_area` 统计

## 直观理解

- **为什么需要它们？**
  - 当肿瘤整体很小（`size` 小），偶然的 1–2 个点噪声会让“连通域数量”虚高，导致 ITH 被错误放大。因此在小样本模式下用 `thresh` 更积极地移除小的连通域。
  - 当肿瘤整体足够大（`size` 大），仅移除面积为 1 的点噪声即可。

## 调参建议

- 如果你的数据噪声多或分辨率高，导致很多小散点：
  - 可适当提高 `thresh`（如 2–5），但注意只在小样本模式下生效。
- 如果你不想区分大小样本，想统一过滤规则：
  - 可将大样本模式的阈值也参数化（把 `<= 1` 改成可配置值）。
- 如果肿瘤体素数分布很不均衡：
  - 调整 `min_area` 以匹配你的数据规模，使“小样本模式”触发点更合理。